In [ ]:
import selenium
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options

import random
from multiprocessing.pool import ThreadPool, Pool

from bs4 import BeautifulSoup 
import re as r
import os
import time
import numpy as np
import csv
import urllib
import cssutils

#this part of the code below chooses a header at random from a collection of 10 headers
header_list=[{'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.157 Safari/537.36COOKIE: 5=2; ax=v167-7'},
       {'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) HeadlessChrome/74.0.3729.157 Safari/537.36'},
       {'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) HeadlessChrome/79.0.3945.88 Safari/537.36'},
       {'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.87 Safari/537.36'},
       {'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.132 Safari/537.36'},
       {'User-Agent':'Mozilla/5.0 (X11; Datanyze; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36'},
       {'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.24 Safari/537.36'}]


def head():
    n = random.random()
    n=int(n*7)
    return header_list[n]

def write_output(data):
    if(not os.path.isfile('link.csv')):
        with open("link.csv", mode="w", newline='',encoding='UTF-8') as out_file:
            writer = csv.writer(out_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_ALL)
            writer.writerow(['link'])
        
    with open("link.csv", mode="a", newline='',encoding='UTF-8') as out_file:
        writer = csv.writer(out_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_ALL)
        writer.writerow(data)


def scrape(address):
    path_chrome = r"chromedriver"
    ua = head()
    user_agent = "user-agent="+ua['User-Agent']
    chrome_options= webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument(user_agent)
    print('user-agent\n',user_agent)
    driver = webdriver.Chrome(path_chrome,options=chrome_options)
    print('opening defualt page')
    url  = 'https://www.google.com/maps/@30.7678096,76.4684188,14z'
    
    driver.get(url)

    element = driver.find_element_by_xpath("//input[@id='searchboxinput']")
    search  = driver.find_element_by_xpath("//button[@id='searchbox-searchbutton']")
    element.clear()
    element.send_keys(address)
    search.click()

    time.sleep(15)
    
    all_img = driver.find_element_by_xpath("//div[@class='section-carouselphoto-photo-container-shim']").click()
    time.sleep(3)
    
    soup = BeautifulSoup(driver.page_source, "lxml")
    
    #images
    results = soup.findAll('div', {'class': 'gallery-image-low-res'})
    print(len(results))
    
    link = [[],[]]
    for i in results:
        i = """{0}""".format(i)
        soup1 = BeautifulSoup(i,'html.parser')
        div_style = soup1.find('div')['style']
        style = cssutils.parseStyle(div_style)
        url = style['background-image'].replace('url(', '').replace(')', '')
        try:
            index = url.find('=')
            url = url[:index] + "=w1080-h1080-k-no"
        except:
            url = "None"
        if("googleusercontent" in url):
            link[0].append(url)
    time.sleep(3)
    
    try:
        menu_btn = driver.find_element_by_xpath('//button[@class="spF7nQ9kj1t__tab"]').click()
        time.sleep(3)
        
        soup = BeautifulSoup(driver.page_source, "lxml")
        #images
        results = soup.find_all('div', {'class': 'gallery-image-low-res'})
        print(len(results))
        for i in results:
            i = """{0}""".format(i)
            soup1 = BeautifulSoup(i,'html.parser')
            div_style = soup1.find('div')['style']
            style = cssutils.parseStyle(div_style)
            url = style['background-image'].replace('url(', '').replace(')', '')
            try:
                index = url.find('=')
                url = url[:index] + "=w1080-h1080-k-no"
            except:
                url = "None"
            if("googleusercontent" in url):
                link[1].append(url)
    except:
        print("menu not found")
    time.sleep(2)
    yield(link)
    
    driver.close()


if __name__=="__main__":
    restaurant_data = pd.read_csv('Restaurants_Data_of_all_Cities_in_British_Columbia.csv')
    restaurant_data['keyword'] = restaurant_data['Name'] +" " + restaurant_data['Address']
    keywords_list = list(restaurant_data['keyword'])
    l = len(keywords_list)
    for i in keywords_list[0:l]:
        try:
            data = scrape(i)
            write_output(data)
        except:
            print(i)